. 
# Performance Optimization

# Parallelism

## Multithreading & Multiprocessing

![](multiprocessing.png)

Python is a hard language to make multithreaded, mostly because of the **global interpreter lock**: Python is an interpreted language, and if the interpreter runs in only one thread, all the nice threads you're producing also run in only one thread.  
While multi*processing* finds a way around that, even multithreading can still be of use. While multithreading can, in Python, not be **parallel**, it can still be **concurrent**. 
* *parallel* processes run truly at the same time - meaning that they must run sumultaneously on different CPU-cores
* *concurrent* processes appear to be parallel to most of the system, even though the CPU may handle them one after another (either parallel or interlocked)

While CPU-intense processes are only truly sped up when they are parallel (something where Python's multithreading doesn't help), tasks that have a bottleneck in network or disk access are helped greatly from cuncurrent execution already.

## Multithreading

To a process, there's a lot of information for the computer:
* State of the process (ready, running, inactive)
* Program-counter (next commans)
* CPU registers (cashed)
* Scheduling information (priority, position)
* Storage information
* I/O-Status
* ...

When switching processes, all those information needs to be saved, such that the CPU can load another process and freeze this one. This produces a lot of overhead. 

**Threads** are lightweight processes, using *shared resources*:
* Shared storage space
* Shared program code
* Shared (virtual) files.

Modern operating systems use threads to let programs switch control, without all the overhead of having to save and load all the information.   
**Advantages**:
* Much faster creation and task switching
* Efficient communication between threads (unlike processes)
* Operating system doesn't schedule them, processes can implement their own scheduling

**Disadvantages**:
* Operating system doesn't schedule them, harder to synchronize than processes
* Processes are better isolated
* Crashing Thread = Crashing Program  
* Python can't use them for parallel processing

In [14]:
import numpy as np
import time

a = np.random.rand(1000000)

def sorter():
    np.sort(a)

start = time.perf_counter()

for i in range(100):
    sorter()

print(time.perf_counter() - start, "seconds")

7.252007194001635 seconds


In [15]:
import threading

a = np.random.rand(1000000)

def sorter():
    np.sort(a)

start = time.perf_counter()

threads = []
for i in range(100):
    t = threading.Thread(target=sorter)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(time.perf_counter() - start, "seconds")

1.5727858919999562 seconds


In [16]:
# %load download.py
import json
import os
from pathlib import Path
from urllib.request import urlopen, Request
from time import time

def get_links(client_id):
   headers = {'Authorization': 'Client-ID {}'.format(client_id)}
   all_ims = []
   for page in ["1", "2", "3", "4", "5", "6"]:
       req = Request('https://api.imgur.com/3/gallery/hot/viral/month', headers=headers, method='GET')
       with urlopen(req) as resp:
           data = json.loads("\n".join([i.decode('utf-8') for i in resp.readlines()]))
       all_ims = all_ims+[i['link'] for i in data['data']]
   return all_ims


def download_link(directory, link):
   download_path = directory / os.path.basename(link)
   with urlopen(link) as image, download_path.open('wb') as f:
       for line in image.readlines():
            f.write(line)

        
def setup_download_dir():
   download_dir = Path('images')
   if not download_dir.exists():
       download_dir.mkdir()
   return download_dir
   



In [17]:
import json
import os
from pathlib import Path
from urllib.request import urlopen, Request
from time import time

In [18]:
from download import *

ts = time()
client_id = 'f8f603617f590ed'
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if l.endswith('.jpg') or l.endswith('.gif') or l.endswith('.mp4')]
for link in links:
   download_link(download_dir, link)
print('Took {}s'.format(time() - ts))

Took 39.50147891044617s


In [19]:
from queue import Queue
from threading import Thread

class DownloadWorker(Thread):
   def __init__(self, queue):
       Thread.__init__(self)
       self.queue = queue

   def run(self):
       while True:
           # Get the work from the queue and expand the tuple
           directory, link = self.queue.get()
           download_link(directory, link)
           self.queue.task_done()

ts = time()
client_id = 'f8f603617f590ed'
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if l.endswith('.jpg') or l.endswith('.gif') or l.endswith('.mp4')]
# Create a queue to communicate with the worker threads
queue = Queue()
# Create 8 worker threads
for x in range(8):
   worker = DownloadWorker(queue)
   # Setting daemon to True will let the main thread exit even though the workers are blocking
   worker.daemon = True
   worker.start()
# Put the tasks into the queue as a tuple
for link in links:
   queue.put((download_dir, link))
# Causes the main thread to wait for the queue to finish processing all the tasks
queue.join()
print('Took {}'.format(time() - ts))

Took 16.887265920639038


Now, there is a class ``DownloadWorker``, which *is* a Python Thread. On every iteration of it's run, it calls ``self.queue.get()``, which fetches a URL from a thread-safe queue. Once the worker recieves such an item, it calls the ``download_link`` method that we used before. Then the worker must signal the queue that the task is done -- if not, ``queue.join()`` would block the main thread forever.  

Note that while this method is concurrent, it is **not parallel** due to Python's GIL! - it is faster because the IO is the bottleneck in this task! The processor is mostly waiting, and can pick up working on a thread as soon as the network is done.

## Multiprocessing 
If code is performing a CPU-heavy task, the execution time will probably be **slower**!  
For such tasks, we need the ``multiprocessing`` module!

To use multiple processing, what we generally do is to create a multiprocessing ``Pool``, which provides a method ``map``. This method is passed a list of URLs to the pool, which spawns individual processes - processes that can execute our download of the images *truly parallel*.  
As mentioned above, the disadvantage of this method is that the entire memory of the script must be copied to every created subprocess, including all its overhead.

In [20]:
from functools import partial
from multiprocessing.pool import Pool

ts = time()
client_id = 'f8f603617f590ed'
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if l.endswith('.jpg') or l.endswith('.gif') or l.endswith('.mp4')]
download = partial(download_link, download_dir)
with Pool(8) as p:
   p.map(download, links)
print('Took {}s'.format(time() - ts))

Took 22.002023458480835s


https://docs.python.org/2/library/multiprocessing.html#using-a-pool-of-workers

## Async/Await

Next to multiprocessing and multithreading for parallel processing, Python also provides the possibility for **asynchronous programming**, a different paradigm for parallel programming, mostly known from Javascript: [https://www.youtube.com/watch?v=3CmKIUmLmJo](https://www.youtube.com/watch?v=3CmKIUmLmJo)  

For an example of that, it is referred to the full version of the source of this code and explanations at [https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python](https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python).

# Numba

In [23]:
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(slope, intercept):
    fig, ax = plt.subplots()
    x = np.linspace(-10, 10, num=1000)
    ax.plot(x, slope * x + intercept)
    ax.set_ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, slope=(-2.0, 2.0), intercept=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='slope', max=2.0, min=-2.0), FloatSlider(value=0.0, d…

In [ ]:
x = np.arange(10_000).reshape(100, 100)

In [ ]:
%%time
go_slow(x)

In [ ]:
%%time 
go_fast(x)

# Cython

* one way to make your program run faster: compile to a faster language.

# Distributed Computing

* second way to make it faster: make a computation graph & distribute.

## Dask
https://dask.org/

tbd